In [23]:
import os
import random
import shutil
import time
import cv2
import glob
import yaml
import matplotlib.pyplot as plt
import ultralytics
import numpy as np
import torch
from torchvision import transforms
from tqdm import tqdm
from PIL import Image
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
data_root = 'D:\\project\\data\\src'
img_root = f'{data_root}\\img'
project_name = 'human_fall'

train_root = f'{data_root}\\{project_name}\\train'
valid_root = f'{data_root}\\{project_name}\\valid'
test_root = f'{data_root}\\{project_name}\\test'

for folder in [train_root, valid_root, test_root] : 
    if not os.path.exists(folder) : 
        os.makedirs(folder)

In [20]:
# train, valid, test 분할
file_list = glob.glob(f'{img_root}\\*\\*.jpg')
#print(len(file_list))

random.shuffle(file_list)
test_ratio = 0.1 
count_files = len(file_list)

train_list = file_list[int(count_files * test_ratio) * 2 : ]
valid_list = file_list[int(count_files * test_ratio) : int(count_files * test_ratio) * 2]
test_list = file_list[ : int(count_files * test_ratio)]

for file in train_list:
    shutil.copy(file, train_root)
for file in valid_list:
    shutil.copy(file, valid_root)
for file in test_list:
    shutil.copy(file, test_root)

In [21]:
data = dict()
class_list = ['나중에 입력']

data['path'] = img_root
data['train'] = train_root
data['val'] = valid_root
data['test'] = test_root
data['nc'] = len(class_list)
data['names'] = class_list

yaml_path = os.path.join(data_root, 'human_fall.yaml')
with open(yaml_path, 'w') as f : 
    yaml.dump(data, f)
print(f'이 경로에 yaml파일 생성 : {yaml_path}')

이 경로에 yaml파일 생성 : D:\project\data\src\human_fall.yaml


In [10]:
!cd D:\project\data\src 

In [24]:
print(f"Train images: {len(os.listdir(train_root))}")
print(f"Valid images: {len(os.listdir(valid_root))}")
print(f"Test images: {len(os.listdir(test_root))}")

Train images: 163200
Valid images: 20400
Test images: 20400


In [26]:
start_time = time.time()

model = YOLO('yolov8l.pt')
result = model.train(data = yaml_path, epochs = 100, batch = 8, imgsz = (960, 540), device = device, workers = 20, amp = False, patience = 30, name = 'human_fall_L')

end_time = time.time()
execution_time = end_time - start_time
print(f"실행 시간: {execution_time:.4f} 초")

New https://pypi.org/project/ultralytics/8.3.7 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8l.pt, data=D:\project\data\src\human_fall.yaml, epochs=100, time=None, patience=30, batch=8, imgsz=(960, 540), save=True, save_period=-1, cache=False, device=cuda, workers=20, project=None, name=human_fall_L10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

train: Scanning D:\project\data\src\human_fall\train... 0 images, 13385 backgrounds, 0 corrupt:   8%|▊         | 13385/163200 [05:54<1:06:07, 37.76it/s]


KeyboardInterrupt: 